In [1]:
import os
import glob
import json
import pandas as pd

In [2]:
metadata = { 'model': [], 'task': [], 'metric': [], 'value': [] }

In [3]:
# Language Tuning
lang_model_paths = glob.glob('./save/*_lang')
for lang_model_path in lang_model_paths:
    lang_model = lang_model_path.split('/')[-1]
    json_path = f'{lang_model_path}/facebook/wav2vec2-large-xlsr-53/all_results.json'
    
    for key, value in json.load(open(json_path,'r')).items():
        if 'eval' not in key:
            continue
        metric = key.split('_')[-1]
        eval_key = '_'.join(key.split('_')[:-1]).split('_nusantara_sptext')[0].split('eval_')[-1]
        if metric in ['cer', 'mer']:
            metadata['model'].append(lang_model)
            metadata['task'].append(eval_key)
            metadata['metric'].append(metric)
            metadata['value'].append(value)

In [4]:
# Indonesian-NLP Language Tuning
indonlp_lang_model_paths = glob.glob('./save/*_lang/indonesian-nlp/*')
for lang_model_path in indonlp_lang_model_paths:
    lang_model = '_'.join([lang_model_path.split('/')[-3], lang_model_path.split('/')[-1]])
    json_path = f'{lang_model_path}/all_results.json'
    
    if not os.path.exists(json_path):
        print(json_path)
        continue
        
    for key, value in json.load(open(json_path,'r')).items():
        if 'eval' not in key:
            continue
        metric = key.split('_')[-1]
        eval_key = '_'.join(key.split('_')[:-1]).split('_nusantara_sptext')[0].split('eval_')[-1]
        if metric in ['cer', 'mer']:
            metadata['model'].append(lang_model)
            metadata['task'].append(eval_key)
            metadata['metric'].append(metric)
            metadata['value'].append(value)

In [5]:
# Task-Specific Tuning
for task_model_path in glob.glob('./save/*'):
    if task_model_path in lang_model_paths:
        continue
    
    model = 'direct_ft'
    json_path = f'{task_model_path}/facebook/wav2vec2-large-xlsr-53/all_results.json'
    if not os.path.exists(json_path):
        print(json_path)
        continue
        
    for key, value in json.load(open(json_path,'r')).items():
        if 'eval' not in key:
            continue            
        metric = key.split('_')[-1]
        eval_key = '_'.join(key.split('_')[:-1]).split('_nusantara_sptext')[0].split('eval_')[-1]
        if metric in ['cer', 'mer']:
            metadata['model'].append(model)
            metadata['task'].append(eval_key)
            metadata['metric'].append(metric)
            metadata['value'].append(value)

./save/su_id_asr/facebook/wav2vec2-large-xlsr-53/all_results.json
./save/titml_idn/facebook/wav2vec2-large-xlsr-53/all_results.json
./save/jv_id_asr/facebook/wav2vec2-large-xlsr-53/all_results.json


In [6]:
# Indonesian-NLP Tuning
for indonlp_model_path in glob.glob('./save/*/indonesian-nlp/*'):
    if indonlp_model_path in indonlp_lang_model_paths:
        continue
        
    model = indonlp_model_path.split('/')[-1]
    json_path = f'{indonlp_model_path}/all_results.json'
    if not os.path.exists(json_path):
        print(json_path)
        continue
        
    for key, value in json.load(open(json_path,'r')).items():
        if 'eval' not in key:
            continue            
        metric = key.split('_')[-1]
        eval_key = '_'.join(key.split('_')[:-1]).split('_nusantara_sptext')[0].split('eval_')[-1]
        if metric in ['cer', 'mer']:
            metadata['model'].append(model)
            metadata['task'].append(eval_key)
            metadata['metric'].append(metric)
            metadata['value'].append(value)

./save/su_id_asr/indonesian-nlp/wav2vec2-large-xlsr-indonesian/all_results.json
./save/su_id_asr/indonesian-nlp/wav2vec2-indonesian-javanese-sundanese/all_results.json
./save/su_id_asr/indonesian-nlp/wav2vec2-large-xlsr-indonesian-baseline/all_results.json
./save/jv_id_asr/indonesian-nlp/wav2vec2-large-xlsr-indonesian/all_results.json
./save/jv_id_asr/indonesian-nlp/wav2vec2-indonesian-javanese-sundanese/all_results.json
./save/jv_id_asr/indonesian-nlp/wav2vec2-large-xlsr-indonesian-baseline/all_results.json


In [7]:
task_to_lang_map = {
	'indspeech_digit_cdsr': 'ind',
	'indspeech_news_ethnicsr_jv_nooverlap': 'jav',
	'indspeech_news_ethnicsr_su_nooverlap': 'sun',
	'indspeech_news_lvcsr': 'ind',
	'indspeech_newstra_ethnicsr_nooverlap_ban': 'ban',
	'indspeech_newstra_ethnicsr_nooverlap_btk': 'btk',
	'indspeech_newstra_ethnicsr_nooverlap_jav': 'jav',
	'indspeech_newstra_ethnicsr_nooverlap_sun': 'sun',
	'indspeech_teldialog_lvcsr': 'ind',
	'indspeech_teldialog_svcsr': 'ind',
	'librivox_indonesia_ace': 'ace',
	'librivox_indonesia_ban': 'ban',
	'librivox_indonesia_bug': 'bug',
	'librivox_indonesia_ind': 'ind',
	'librivox_indonesia_jav': 'jav',
	'librivox_indonesia_min': 'min',
	'librivox_indonesia_sun': 'sun'
}

In [8]:
df = pd.DataFrame(metadata)
df = df[df['task'] != 'indspeech_news_tts_120_MOS']
df.loc[:,'lang'] = df.loc[:,'task'].apply(lambda x: task_to_lang_map[x])

In [9]:
df.groupby('model').size()

model
all_lang                                           34
all_lang_wav2vec2-indonesian-javanese-sundanese    34
direct_ft                                          34
ind_lang                                           34
ind_lang_wav2vec2-indonesian-javanese-sundanese    34
jav_lang                                           34
jav_lang_wav2vec2-indonesian-javanese-sundanese    34
sun_lang                                           34
sun_lang_wav2vec2-indonesian-javanese-sundanese    34
wav2vec2-indonesian-javanese-sundanese             34
wav2vec2-large-xlsr-indonesian                     34
wav2vec2-large-xlsr-indonesian-baseline            34
dtype: int64

In [10]:
df[df['metric'] == 'cer'].groupby(['model'])['value'].mean()

model
all_lang                                           0.114607
all_lang_wav2vec2-indonesian-javanese-sundanese    0.041367
direct_ft                                          0.496622
ind_lang                                           0.276927
ind_lang_wav2vec2-indonesian-javanese-sundanese    0.929311
jav_lang                                           0.289619
jav_lang_wav2vec2-indonesian-javanese-sundanese    0.256416
sun_lang                                           0.309507
sun_lang_wav2vec2-indonesian-javanese-sundanese    0.308342
wav2vec2-indonesian-javanese-sundanese             0.061922
wav2vec2-large-xlsr-indonesian                     0.077544
wav2vec2-large-xlsr-indonesian-baseline            0.090485
Name: value, dtype: float64

In [11]:
df[df['metric'] == 'mer'].groupby(['model'])['value'].mean()

model
all_lang                                           0.170342
all_lang_wav2vec2-indonesian-javanese-sundanese    0.197625
direct_ft                                          0.633924
ind_lang                                           1.015505
ind_lang_wav2vec2-indonesian-javanese-sundanese    2.977098
jav_lang                                           0.759816
jav_lang_wav2vec2-indonesian-javanese-sundanese    0.617633
sun_lang                                           0.823687
sun_lang_wav2vec2-indonesian-javanese-sundanese    0.757558
wav2vec2-indonesian-javanese-sundanese             0.264071
wav2vec2-large-xlsr-indonesian                     0.363372
wav2vec2-large-xlsr-indonesian-baseline            0.403083
Name: value, dtype: float64

In [12]:
df[df['metric'] == 'cer'].groupby(['model', 'lang'])['value'].mean().reset_index().pivot('model','lang','value')

lang,ace,ban,btk,bug,ind,jav,min,sun
model,,,,,,,,
all_lang,0.272503,0.098403,0.131101,0.281804,0.066198,0.119640,0.108794,0.089133
all_lang_wav2vec2-indonesian-javanese-sundanese,0.074349,0.042029,0.063187,0.085779,0.007279,0.066862,0.028916,0.043321
direct_ft,0.967782,0.498294,0.134273,0.924755,0.098513,0.701887,0.890763,0.643396
ind_lang,0.363073,0.428657,0.890919,0.253574,0.014567,0.418357,0.081124,0.311283
ind_lang_wav2vec2-indonesian-javanese-sundanese,0.350682,1.635763,4.080932,0.325056,0.006464,1.324080,0.162249,1.201094
jav_lang,0.403965,0.143335,0.178914,0.265613,0.581454,0.103313,0.116466,0.151562
jav_lang_wav2vec2-indonesian-javanese-sundanese,0.371747,0.117872,0.136674,0.300978,0.534302,0.068637,0.106024,0.110159
sun_lang,0.311029,0.182998,0.132707,0.288187,0.594965,0.267269,0.176707,0.070121
sun_lang_wav2vec2-indonesian-javanese-sundanese,0.338290,0.189617,0.118720,0.376975,0.587157,0.245169,0.205622,0.050560


In [13]:
df[df['metric'] == 'mer'].groupby(['model', 'lang'])['value'].mean().reset_index().pivot('model','lang','value')

lang,ace,ban,btk,bug,ind,jav,min,sun
model,,,,,,,,
all_lang,0.408547,0.167284,0.189804,0.415906,0.080493,0.185719,0.169406,0.139321
all_lang_wav2vec2-indonesian-javanese-sundanese,0.319444,0.210536,0.359930,0.533019,0.019010,0.275473,0.180952,0.207913
direct_ft,1.000000,0.719914,0.647735,1.000000,0.125099,0.857790,1.000000,0.830093
ind_lang,0.951389,1.664475,2.971777,0.966981,0.042044,1.499459,0.461905,1.291327
ind_lang_wav2vec2-indonesian-javanese-sundanese,0.916667,5.445443,12.403310,1.132075,0.018721,4.120569,0.704762,4.035885
jav_lang,1.034722,0.670187,0.812369,1.099057,0.888710,0.469665,0.680952,0.698948
jav_lang_wav2vec2-indonesian-javanese-sundanese,0.902778,0.526303,0.597909,1.028302,0.788654,0.272313,0.528571,0.543127
sun_lang,0.923611,0.823727,0.746690,1.122642,0.912212,0.934332,0.985714,0.404170
sun_lang_wav2vec2-indonesian-javanese-sundanese,0.895833,0.765177,0.613415,1.160377,0.895944,0.885029,0.790476,0.251076


In [14]:
df.groupby('model').size()

model
all_lang                                           34
all_lang_wav2vec2-indonesian-javanese-sundanese    34
direct_ft                                          34
ind_lang                                           34
ind_lang_wav2vec2-indonesian-javanese-sundanese    34
jav_lang                                           34
jav_lang_wav2vec2-indonesian-javanese-sundanese    34
sun_lang                                           34
sun_lang_wav2vec2-indonesian-javanese-sundanese    34
wav2vec2-indonesian-javanese-sundanese             34
wav2vec2-large-xlsr-indonesian                     34
wav2vec2-large-xlsr-indonesian-baseline            34
dtype: int64

In [15]:
df.pivot(['model'], ['task', 'metric'], 'value').reset_index().to_csv('asr_result.csv', index=False)

In [16]:
df.pivot(['model'], ['task', 'metric'], 'value')

task                                            indspeech_digit_cdsr  \
metric                                                           cer   
model                                                                  
all_lang                                                    0.023752   
all_lang_wav2vec2-indonesian-javanese-sundanese             0.003817   
direct_ft                                                   0.438411   
ind_lang                                                    0.002154   
ind_lang_wav2vec2-indonesian-javanese-sundanese             0.001821   
jav_lang                                                    0.975105   
jav_lang_wav2vec2-indonesian-javanese-sundanese             0.928639   
sun_lang                                                    0.976514   
sun_lang_wav2vec2-indonesian-javanese-sundanese             0.978212   
wav2vec2-indonesian-javanese-sundanese                      0.002170   
wav2vec2-large-xlsr-indonesian                              0.014063   
wav2vec2-large-xlsr-indonesian-baseline                     0.014092   

task                                                       \
metric                                                mer   
model                                                       
all_lang                                         0.023632   
all_lang_wav2vec2-indonesian-javanese-sundanese  0.003817   
direct_ft                                        0.438411   
ind_lang                                         0.002154   
ind_lang_wav2vec2-indonesian-javanese-sundanese  0.001821   
jav_lang                                         0.975105   
jav_lang_wav2vec2-indonesian-javanese-sundanese  0.928629   
sun_lang                                         0.976514   
sun_lang_wav2vec2-indonesian-javanese-sundanese  0.974624   
wav2vec2-indonesian-javanese-sundanese           0.002170   
wav2vec2-large-xlsr-indonesian                   0.022383   
wav2vec2-large-xlsr-indonesian-baseline          0.022356   

task                                            indspeech_news_ethnicsr_jv_nooverlap  \
metric                                                                           cer   
model                                                                                  
all_lang                                                                    0.081622   
all_lang_wav2vec2-indonesian-javanese-sundanese                             0.049470   
direct_ft                                                                   1.000000   
ind_lang                                                                    0.369763   
ind_lang_wav2vec2-indonesian-javanese-sundanese                             0.509844   
jav_lang                                                                    0.085815   
jav_lang_wav2vec2-indonesian-javanese-sundanese                             0.043412   
sun_lang                                                                    0.240787   
sun_lang_wav2vec2-indonesian-javanese-sundanese                             0.226148   
wav2vec2-indonesian-javanese-sundanese                                      0.059818   
wav2vec2-large-xlsr-indonesian                                              0.089349   
wav2vec2-large-xlsr-indonesian-baseline                                     0.101212   

task                                                       \
metric                                                mer   
model                                                       
all_lang                                         0.133707   
all_lang_wav2vec2-indonesian-javanese-sundanese  0.223011   
direct_ft                                        1.000000   
ind_lang                                         1.390625   
ind_lang_wav2vec2-indonesian-javanese-sundanese  1.517045   
jav_lang                                         0.375000   
jav_lang_wav2vec2-indonesian-javanese-sundanese  0.178977   
sun_lang                                         0.933239   
sun_lang_wa

In [17]:
df[df['metric'] == 'cer'].pivot(['model'], ['task'], 'value').reset_index().to_csv('asr_result_cer.csv', index=False)

In [18]:
df[df['metric'] == 'mer'].pivot(['model'], ['task'], 'value').reset_index().to_csv('asr_result_mer.csv', index=False)

In [19]:
df[df['metric'] == 'cer'].groupby(['model', 'lang'])['value'].mean().reset_index().pivot('model','lang','value').reset_index().to_csv('asr_result_cer_lang.csv', index=False)

In [20]:
df[df['metric'] == 'mer'].groupby(['model', 'lang'])['value'].mean().reset_index().pivot('model','lang','value').reset_index().to_csv('asr_result_mer_lang.csv', index=False)